Here I only test parts of the code before passing them on to the main notebook.

In [2]:
import time
import requests
from dotenv import load_dotenv
import os
import folium
import pandas as pd
import json

In [6]:
m = folium.Map(location=[20.9939879883004, -89.62853393602846], zoom_start=12, min_zoom=12, max_zoom=12)
delta = 0.0075

initial_lat, initial_lng = 20.944451,-89.654108 


rectangles_viewports = []

for i in range(8):
    for j in range(8):
        low = [initial_lat+i*delta, initial_lng+j*delta]
        high = [initial_lat+(i+1)*delta, initial_lng+(j+1)*delta]
        folium.Rectangle(
            bounds = [low, high],
            tooltip = f'({i+1},{j+1})',
            fill = True
        ).add_to(m)
        
        rectangles_viewports.append((tuple(low), tuple(high)))
m

In [9]:
print(rectangles_viewports)
print(len(rectangles_viewports))

[((20.944451, -89.654108), (20.951951, -89.646608)), ((20.944451, -89.646608), (20.951951, -89.639108)), ((20.944451, -89.639108), (20.951951, -89.631608)), ((20.944451, -89.631608), (20.951951, -89.62410799999999)), ((20.944451, -89.62410799999999), (20.951951, -89.616608)), ((20.944451, -89.616608), (20.951951, -89.60910799999999)), ((20.944451, -89.60910799999999), (20.951951, -89.601608)), ((20.944451, -89.601608), (20.951951, -89.59410799999999)), ((20.951951, -89.654108), (20.959451, -89.646608)), ((20.951951, -89.646608), (20.959451, -89.639108)), ((20.951951, -89.639108), (20.959451, -89.631608)), ((20.951951, -89.631608), (20.959451, -89.62410799999999)), ((20.951951, -89.62410799999999), (20.959451, -89.616608)), ((20.951951, -89.616608), (20.959451, -89.60910799999999)), ((20.951951, -89.60910799999999), (20.959451, -89.601608)), ((20.951951, -89.601608), (20.959451, -89.59410799999999)), ((20.959451, -89.654108), (20.966951, -89.646608)), ((20.959451, -89.646608), (20.96695

In [10]:
load_dotenv()
API_KEY = os.getenv('API_KEY')

search_url = "https://places.googleapis.com/v1/places:searchText"
headers = {
    'Content-Type' : 'application/json',
    'X-Goog-Api-Key': API_KEY,
    'X-Goog-FieldMask': 'places.id'
}

In [11]:
raw_data = []
for low, high in rectangles_viewports:
    try: 
        payload = {
            'textQuery' : 'cafeteria',
            'includedType': 'cafe',
            'strictTypeFiltering':True,
            'locationRestriction' : {
                'rectangle':{
                    'low':{
                        'latitude' : low[0],
                        'longitude' : low[1]
                    },
                    'high':{
                        'latitude': high[0],
                        'longitude': high[1]
                        
                    }
                }
            }
        }
        
        response = requests.post(url = search_url, json=payload, headers = headers)
        response.raise_for_status()
        
        data = response.json()
        
        raw_data.extend(data.get('places', []))
        
    except requests.exceptions.RequestException as e:
        print(f'ERROR!!! --> {e}')
    
    time.sleep(0.5)

In [12]:
with open('places_ids.json', 'w') as f:
    json.dump(raw_data, f, indent=4)

In [13]:
id_df = pd.read_json('places_ids.json')
id_df

,id
0,ChIJlSpZ2wlzVo8RdsMWLOZqrg8
1,ChIJ60iXwWlzVo8RQ-Dfk5clkr4
2,ChIJQW2PaABzVo8RHhAxixzJRro
3,ChIJ8w3ISABxVo8R3oEaIx4Bl8Q
4,ChIJFcULupdzVo8Rv8bw9jY2rr8
...,...
191,ChIJR_TI1NJ2Vo8RtQbjdP4OgmA
192,ChIJMTs540N3Vo8RFJ24m6TEo3k
193,ChIJxw8NofB3Vo8RY9OTbZVIWQE
194,ChIJQXUxgSJ3Vo8RSo8q8fkI5bI
